<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#scrap-championships" data-toc-modified-id="scrap-championships-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>scrap championships</a></span></li><li><span><a href="#scrap-teams-and-players" data-toc-modified-id="scrap-teams-and-players-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>scrap teams and players</a></span></li><li><span><a href="#scrap-matches" data-toc-modified-id="scrap-matches-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>scrap matches</a></span></li><li><span><a href="#add-meteo-to-the-df_matches" data-toc-modified-id="add-meteo-to-the-df_matches-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>add meteo to the df_matches</a></span></li><li><span><a href="#scrap-goals-par-player/match" data-toc-modified-id="scrap-goals-par-player/match-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>scrap goals par player/match</a></span></li><li><span><a href="#add-df-to-sql-database" data-toc-modified-id="add-df-to-sql-database-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>add df to sql database</a></span><ul class="toc-item"><li><span><a href="#create-championship-table-sql" data-toc-modified-id="create-championship-table-sql-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>create championship table sql</a></span></li><li><span><a href="#create-teams-table-in-sql" data-toc-modified-id="create-teams-table-in-sql-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>create teams table in sql</a></span></li><li><span><a href="#create-participation-table" data-toc-modified-id="create-participation-table-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>create participation table</a></span></li><li><span><a href="#create-players-table" data-toc-modified-id="create-players-table-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>create players table</a></span></li><li><span><a href="#create-matches-table" data-toc-modified-id="create-matches-table-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>create matches table</a></span></li><li><span><a href="#create-matches_teams-table" data-toc-modified-id="create-matches_teams-table-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>create matches_teams table</a></span></li><li><span><a href="#create-goals-table" data-toc-modified-id="create-goals-table-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>create goals table</a></span></li></ul></li></ul></div>

In [8]:
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns

In [9]:
url = "https://www.lequipe.fr/Football/"
main_url = 'https://www.lequipe.fr'

In [10]:
# function to get page soup from html page
import requests
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq

def get_page_soup(url, time = 15):
    """get_page_soup(url) takes url of the web page and return it html soup.
    Parameters:
    url: <str> web page address
    time: <int> maximum time in seconds to read page
    Output:
    BeautifulSoup()"""
    # opening connection
    uClient = uReq(url, timeout=time)
    page_html = uClient.read()
    uClient.close()
    #html parser
    return soup(page_html, "html.parser")

### scrap championships

In [13]:
#get championships links
main_soup = get_page_soup(url)
#equipe_soup
#a aria-label = '/Football/' class = 'Link Menu__accordionTitle'
championships = main_soup.find("div", {"class": 'Menu__accordion js-menu-accordion is-active'}).findAll("li", {"class":"Menu__item" })
print(championships[0].a['href'])
championships_dict = {'name':[], 'link':[]}
for champ in championships:
    championships_dict['name'].append(champ.a.text.split('\n')[1].strip())
    championships_dict['link'].append(champ.a['href'])

df_championships = pd.DataFrame(championships_dict)
df_championships.head()

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [12]:
results_link_list = []; teams_link_list = []; matches_link_list = []; archives_link_list = []

for i in range(len(df_championships)):
    url_champ = df_championships.loc[i,'link']
    # insert code below here after
    
    # for single championship
    #url_champ = df_championships.loc[2,'link']
    page_championship = get_page_soup(url_champ)
    navigator_items = page_championship.find("div", {"class": "MainNav__items js-mainNavItems"}).findAll("a", {"class": "Link MainNav__item"})

    results_link = None; teams_link = None; matches_link = None; archives_link = None

    for item in navigator_items:

        if "résultats" in item.text.strip().split():
            #print("res")
            results_link = item["href"]
        if "sélections" in item.text.strip().split() or "clubs" in item.text.strip().split():
            #print('TEAMS link: {}'.format(item["href"]))   
            teams_link = item["href"]
        if "matches" in item.text.strip().split() or "matchs" in item.text.strip().split():
            #print('MATCHES link: {}'.format(item["href"]))
            matches_link = item['href']
        if "archives" in item.text.strip().split():
            #print("res")
            archives_link = item["href"]

    #results_link_list.append(results_link)
    teams_link_list.append(teams_link)
    #matches_link_list.append(matches_link)
    archives_link_list.append(archives_link)

#df_championships['result_link'] = results_link_list
df_championships['teams_link'] = teams_link_list
#df_championships['matches_link'] = matches_link_list
df_championships['archives_link'] = archives_link_list

NameError: name 'df_championships' is not defined

In [6]:
df_championships.isnull().sum()

name              0
link              0
teams_link       14
archives_link    15
dtype: int64

### scrap teams and players



In [7]:
## create database teams

In [8]:
# for i in range(len(df_championships)):
#     url_teams = df_championships.loc[i,'teams_link']
#     if url_teams is not None:
#         try:
            
#         except:
#             pass

url_teams = df_championships.loc[2,'teams_link']
champ_name = df_championships.loc[2, 'name']
## champ_year_start = df_championships.loc[2, 'year_start']

teams_page = get_page_soup(url_teams)

teams = teams_page.find("ul", {"class": "clubs"}).findAll("li")
team_links = [team.h2.a['href'] for team in teams if team.h2 is not None]
team_links = [main_url + link for link in team_links if 'https://' not in link ]
team_names = [team.h2.a.text.strip() for team in teams if team.h2 is not None]


In [9]:
# get info for each team

dict_teams = {
    'city': [],
    'name': [],
    'coach_name': [],
    'president': [],
    'year_founded': [],
    'championship': []
}
dict_players = {
    'number': [],
    'name': [],
    'country': [],
    'birthday': [],
    'position': [],
    'club': []
}
df_players = pd.DataFrame(dict_players)
#print(df_players.head())
for i, (link, name) in enumerate(zip(team_links, team_names)):

#     link = df_teams_links.loc[i, 'links']
#     name = df_teams_links.loc[i, 'team_short_names']
    #print(name)
    # team info and player info dirty tables
    team_info = pd.read_html(link)[0]

    dict_teams['city'].append(name)
    dict_teams['name'].append(team_info.loc[0, 0].split(':')[1].strip())
    dict_teams['year_founded'].append(team_info.loc[1, 0].split(':')[1].strip())
    dict_teams['president'].append(team_info.loc[2, 0].split(':')[1].strip())
    dict_teams['coach_name'].append(team_info.loc[3, 0].split(':')[1].strip())
    dict_teams['championship'].append(champ_name)

    # players info
    if len(pd.read_html(link)[2]) > 5:
        team_players = pd.read_html(link)[2]
    else:
        team_players = pd.read_html(link)[1]

    old_column_names = team_players.columns
    team_players = team_players.rename(
        columns={
            'Unnamed: 0': 'number',
            'Nom': 'name',
            'Pays': 'country',
            old_column_names[3]: 'birthday',
            'Pos.': 'position'
        })
    team_players['club'] = name

    df_players = pd.concat([df_players, team_players], ignore_index=True)
    #print(df_players.tail(2))

df_teams = pd.DataFrame(dict_teams)

In [10]:
df_players.isnull().sum()

number      106
name          0
country       0
birthday      5
position      0
club          0
dtype: int64

In [11]:
def change_date_format(a):
    if a is not np.nan:
        a = a.split("/")
        a[-1] = str(int(a[-1])+2000) if int(a[-1]) < 30 else str(int(a[-1]) +1900)
        a[-2] = a[-2].zfill(2)
        a[-3] = a[-3].zfill(2)
        a.reverse()
        a = '-'.join(a)
    return(a)

df_players['birthday'] = df_players['birthday'].apply(change_date_format)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
#df_players['name'].unique()

In [13]:
df_players['birthday'].tail(4)

624    1992-02-08
625    2001-12-20
626    1991-09-19
627    1996-12-19
Name: birthday, dtype: object

In [14]:
df_teams.tail(3)

,city,name,coach_name,president,year_founded,championship
17,Rennes,Stade Rennais Football Club,Bruno GENESIO,Nicolas Holveck,1901,Ligue 1
18,Saint-Étienne,Association Sportive de St-Etienne,Claude PUEL,B. Caïazzo & R. Romeyer,1919,Ligue 1
19,Strasbourg,Racing Club de Strasbourg Alsace,Julien STEPHAN,Marc Keller,1906,Ligue 1


In [15]:
df_players.head(3)


,number,name,country,birthday,position,club
0,1.0,P. Bernardoni,FRA,1997-04-18,Gar.,Angers
1,16.0,L. Butelle,FRA,1983-04-03,Gar.,Angers
2,30.0,D. Petkovic,MNE,1993-05-25,Gar.,Angers


### scrap matches 

In [16]:
# scrap archive to get matches information for each championship
archives_url = df_championships.loc[2,'archives_link']
print(archives_url)

https://www.lequipe.fr/Football/archives/championnat-de-france.html


In [23]:
#get links for all archive of each year of this championship
archives_page = get_page_soup(archives_url)
archives_list = archives_page.findAll("td")
for archive in archives_list:
    ligue_name = archive.h2.text
    print(ligue_name)
    link = archive.findAll('div', {"class":"ligneee"})[1].a['href']
    print(link)

Ligue 1 2020
/Football/ligue-1/saison-2019-2020/page-calendrier-resultats
Ligue 1 2019
/Football/ligue-1/saison-2018-2019/page-calendrier-resultats
Ligue 1 2018
/Football/ligue-1/saison-2017-2018/page-calendrier-resultats
Ligue 1 2017
/Football/ligue-1/saison-2016-2017/page-calendrier-resultats
Ligue 1 2016
/Football/ligue-1/saison-2015-2016/page-calendrier-resultats
Ligue 1 2015
/Football/ligue-1/saison-2014-2015/page-calendrier-resultats
Ligue 1 2014
/Football/ligue-1/saison-2013-2014/page-calendrier-resultats
Ligue 1 2013
/Football/ligue-1/saison-2012-2013/page-calendrier-resultats
Ligue 1 2012
/Football/ligue-1/saison-2011-2012/page-calendrier-resultats
Ligue 1 2011
/Football/ligue-1/saison-2010-2011/page-calendrier-resultats
Ligue 1 2010
/Football/ligue-1/saison-2009-2010/page-calendrier-resultats
Ligue 1 2009
/Football/ligue-1/saison-2008-2009/page-calendrier-resultats
Ligue 1 2008
/Football/ligue-1/saison-2007-2008/page-calendrier-resultats
Ligue 1 2007
/Football/ligue-1/saison-

In [24]:
link_matches = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
matches_page = get_page_soup(link_matches)

matches_json = []
matches = matches_page.find("select",{"class":"SelectNav__select"}).findAll('option')
for match in matches:
    matches_json.append(match['value'])

In [25]:
#data from one json table
dict_matches = {
    'date': [],
    'place': [],
    'team_domicile': [],
    'team_exterieur': [],
    'score_domicile': [],
    'score_exterieur': [],
    'vainqueur': []
}


import json
import requests

for match_json in matches_json:
    response = requests.get(match_json)
    data = json.loads(response.text)
    for day in data['items'][:-1]:
        
        for team in day['items']:
            dict_matches["place"].append(team['event']['lieu']['ville'])
            dict_matches["date"].append(team['date'])
            dict_matches["team_domicile"].append(team['event']['specifics']['domicile']['equipe'][
                'nom'])
            dict_matches["team_exterieur"].append(team['event']['specifics']['exterieur']['equipe'][
                'nom'])
            dict_matches["score_domicile"].append(team['event']['specifics']['score']['domicile'])
            dict_matches["score_exterieur"].append(team['event']['specifics']['score']['exterieur'])
            dict_matches["vainqueur"].append(team['event']['specifics']['vainqueur'])

            #print("Stade:{} Date:{}".format(lieu, date))
            #print("{} {}:{} {}".format(team_domicile, score_domicile, score_exterieur, team_exterieur))
            #print("\n")
            
            
df_matches_score = pd.DataFrame(dict_matches)

In [26]:
# from datetime import datetime
# df_matches_score['date']= [ datetime.strptime(date, '%Y-%m-%dT%H:%M:%S%z') for date in df_matches_score['date'] ]

### add meteo to the df_matches

### scrap goals par player/match

In [31]:
#import urllib.request
#from bs4 import BeautifulSoup
import csv
import re
import json
from http.client import IncompleteRead
import numpy as np

url_match_list = []
url = ''
for i in np.arange(1, 39):
    if i == 1:
        url = f'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats/{i}re-journee'
    else:
        url = f'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats/{i}e-journee'

    page_soup = get_page_soup(url)
    matches_json = []
    main = page_soup.find('div', attrs={'class': 'container__main'})
    teams_html = main.findAll('div', attrs={'class': 'TeamScore__data'})
    for l in teams_html:
        link_teams = l.find('a', attrs={'class': 'Link'}, href=True)
        a = link_teams['href']
        url_match_list.append(a)

In [34]:
home_team_goals = []
exterior_team_goals = []
for url in url_match_list:

    page_soup = get_page_soup(url, 20)
    goal_list = page_soup.findAll(
        'div', attrs={'class': 'grid__content Scoreboard__goalList'})

    goal_list_right = []
    goal_list_left = []

    if goal_list == []:
        home_team_goals.append([])
        exterior_team_goals.append([])
    else:

        for s in goal_list[0].findAll('span',
                                      attrs={'class': 'Scoreboard__scorer'}):

            goal_list_left.append(s.text)

        for s in goal_list[1].findAll('span',
                                      attrs={'class': 'Scoreboard__scorer'}):

            goal_list_right.append(s.text)

        home_team_goals.append(goal_list_left)
        exterior_team_goals.append(goal_list_right)

In [35]:
# create database
exterior_team_goals

[[],
 ['I. Traoré'],
 ['D. Da Silva'],
 ['B. Dia', 'E. Touré'],
 ['M. Chahiri'],
 [],
 ['G. Kakuta'],
 ['M. Depay'],
 [],
 ['R. Hamouma', 'D. Bouanga'],
 ['A. Scheidler'],
 ['G. Laborde'],
 ['K. Dolberg', 'K. Dolberg'],
 ['J. Bamba'],
 ['Z. Ferhat'],
 [],
 ['J. Maja', 'T. Basic'],
 ['B. Badiashile'],
 ['F. Thauvin', 'D. Caleta-Car', 'D. Caleta-Car'],
 [],
 [],
 ['Dante'],
 [],
 [],
 ['S. Guirassy', 'S. Guirassy', 'N. Aguerd', 'B. Bourigeaud'],
 ['G. Kakuta', 'F. Medina', 'I. Ganago'],
 [],
 ['R. Perraud', 'I. Cardona'],
 ['L. Blas'],
 ['F. Thauvin'],
 [],
 ['S. Kalu'],
 ['W. Ben Yedder'],
 ['K. Mbappé', 'A. Di Maria', 'Marquinhos'],
 ['S. Bahoken'],
 ['C. Hérelle', 'T. Monconduit'],
 ['B. Dia'],
 [],
 ['A. Aouchiche', 'Y. Maçon'],
 ['L. Araujo'],
 [],
 ['N. Aguerd', 'S. Guirassy', 'A. Hunou'],
 ['I. Niane'],
 [],
 ['A. Delort', 'T. Savanier'],
 ['I. Ganago'],
 ['M. Chahiri', 'L. Ajorque'],
 ['S. Mounié', 'G. Charbonnier'],
 ['L. Dubois'],
 ['M. Icardi', 'M. Icardi'],
 ['I. Traoré'],
 [

In [36]:
df_matches_score['goals_domicile'] = home_team_goals

In [37]:
df_matches_score['goals_exterieur'] = exterior_team_goals

In [106]:
pd.set_option('display.max_rows', None)
display(df_matches_score)

,date,place,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,goals_domicile,goals_exterieur
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,Nantes,0,0,ex-aequo,[],[]
1,2020-08-22T17:00:00+0200,Dijon,Dijon,Angers,0,1,exterieur,[],[I. Traoré]
2,2020-08-22T21:00:00+0200,Lille,Lille,Rennes,1,1,ex-aequo,[J. Bamba],[D. Da Silva]
3,2020-08-23T13:00:00+0200,Monaco,Monaco,Reims,2,2,ex-aequo,"[A. Disasi, B. Badiashile]","[B. Dia, E. Touré]"
4,2020-08-23T15:00:00+0200,Lorient,Lorient,Strasbourg,3,1,domicile,"[Y. Wissa, A. Grbic, P. Hamel]",[M. Chahiri]
5,2020-08-23T15:00:00+0200,Nîmes,Nîmes,Brest,4,0,domicile,"[K. Denkey, B. Meling, R. Philippoteaux, M. Koné]",[]
6,2020-08-23T17:00:00+0200,Nice,Nice,Lens,2,1,domicile,"[A. Gouiri, A. Gouiri]",[G. Kakuta]
7,2020-09-15T21:00:00+0200,Montpellier,Montpellier,Lyon,2,1,domicile,"[T. Savanier, T. Savanier]",[M. Depay]
8,2020-09-16T21:00:00+0200,Paris,Paris-SG,Metz,1,0,domicile,[J. Draxler],[]
9,2020-09-17T21:00:00+0200,Marseille,Marseille,Saint-Étienne,0,2,exterieur,[],"[R. Hamouma, D. Bouanga]"


In [39]:
display(df_matches_score.head())
# df_matches
df_matches = df_matches_score[['date','place']]
df_matches.is_copy = None
display(df_matches.head())

# df_matches_teams
df_matches_teams1 = df_matches_score[['date','place','team_domicile','score_domicile', 'goals_domicile']].rename(columns = {'team_domicile': 'team', 'score_domicile': 'team_goal', 'goals_domicile': 'players'})
df_matches_teams2 = df_matches_score[['date','place','team_exterieur','score_exterieur', 'goals_exterieur']].rename(columns = {'team_exterieur': 'team', 'score_exterieur': 'team_goal', 'goals_exterieur': 'players'})
df_matches_teams = pd.concat([df_matches_teams1, df_matches_teams2], ignore_index=True)
df_matches_teams = df_matches_teams.sort_values(by='date').reset_index(drop=True)
display(df_matches_teams.head(10))   

,date,place,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,goals_domicile,goals_exterieur
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,Nantes,0,0,ex-aequo,[],[]
1,2020-08-22T17:00:00+0200,Dijon,Dijon,Angers,0,1,exterieur,[],[I. Traoré]
2,2020-08-22T21:00:00+0200,Lille,Lille,Rennes,1,1,ex-aequo,[J. Bamba],[D. Da Silva]
3,2020-08-23T13:00:00+0200,Monaco,Monaco,Reims,2,2,ex-aequo,"[A. Disasi, B. Badiashile]","[B. Dia, E. Touré]"
4,2020-08-23T15:00:00+0200,Lorient,Lorient,Strasbourg,3,1,domicile,"[Y. Wissa, A. Grbic, P. Hamel]",[M. Chahiri]


,date,place
0,2020-08-21T19:00:00+0200,Bordeaux
1,2020-08-22T17:00:00+0200,Dijon
2,2020-08-22T21:00:00+0200,Lille
3,2020-08-23T13:00:00+0200,Monaco
4,2020-08-23T15:00:00+0200,Lorient


,date,place,team,team_goal,players
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,0,[]
1,2020-08-21T19:00:00+0200,Bordeaux,Nantes,0,[]
2,2020-08-22T17:00:00+0200,Dijon,Dijon,0,[]
3,2020-08-22T17:00:00+0200,Dijon,Angers,1,[I. Traoré]
4,2020-08-22T21:00:00+0200,Lille,Rennes,1,[D. Da Silva]
5,2020-08-22T21:00:00+0200,Lille,Lille,1,[J. Bamba]
6,2020-08-23T13:00:00+0200,Monaco,Monaco,2,"[A. Disasi, B. Badiashile]"
7,2020-08-23T13:00:00+0200,Monaco,Reims,2,"[B. Dia, E. Touré]"
8,2020-08-23T15:00:00+0200,Lorient,Lorient,3,"[Y. Wissa, A. Grbic, P. Hamel]"
9,2020-08-23T15:00:00+0200,Nîmes,Brest,0,[]


In [40]:
df_matches['temperature'] = [18]*len(df_matches)
df_matches['rainfall'] = [0]*len(df_matches)
df_matches.tail()
print(len(df_matches_score))

380


<ipython-input-40-8d71e27cafc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['temperature'] = [18]*len(df_matches)
<ipython-input-40-8d71e27cafc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['rainfall'] = [0]*len(df_matches)


### add df to sql database 

In [1]:
import sqlite3
class ChampionshipsDataBase:


    def __init__(self, name='championships.sqlite'):
        self.name_db = name

        if os.path.isfile(
                self.name_db):  # if sqlite file exist create a connection
            self.connection = self.__connect_db()
            self.cur = self.connection.cursor()
            print(self.connection)

        else:  # if file sqlite doesn't exist create file and then create sql

            open(self.name_db, 'x')
            self.connection = self.__connect_db()
            print(self.connection)
            self.cur = self.connection.cursor()
            self.__create_db()
            #self.fill_db()

        

    def get_tables_names(self):
        self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

        #store all names of the tables/chemas in var tables
        tables0 = self.cur.fetchall()

        tables = []
        for tab in tables0:
            tables.append(tab[0])

        return tables
    

    def get_df(self, table_name):
        return pd.read_sql_query(f"SELECT * FROM {table_name}",
                                 self.connection)

    def __connect_db(self):
        return sqlite3.connect(self.name_db)
    
    def __create_db(self):
        
        conn = self.connection
        cursor = self.cur

        # cursor.execute('DROP TABLE championships;')
        # cursor.execute('DROP TABLE matches;')
        # cursor.execute('DROP TABLE teams;')
        # cursor.execute('DROP TABLE matches_teams;')
        # cursor.execute('DROP TABLE players;')
        # cursor.execute('DROP TABLE goals;')

        #championships
        cursor.execute("""
        CREATE TABLE championships(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        country VARCHAR(255),
        start_year INTEGER,
        end_year INTEGER
        )
        ;""")

        conn.commit()
           
        #teams
        cursor.execute("""
        CREATE TABLE teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        city VARCHAR(255),
        coach_name VARCHAR(255),
        year_founded INTEGER
        )
        ;""")
        conn.commit()
               
        #participation
        cursor.execute("""
        CREATE TABLE participation(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        championship_id INTEGER,
        team_id INTEGER,
        FOREIGN KEY (championship_id)
        REFERENCES championships(id),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        
        #matches
        cursor.execute("""
        CREATE TABLE matches(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE,
        place VARCHAR(255),
        rainfall REAL,
        temperature REAL
        )
        ;""")
        conn.commit()


        #matches_teams
        cursor.execute("""
        CREATE TABLE matches_teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        match_id INTEGER,
        team_id INTEGER,
        home INTEGER,
        team_goal INTEGER,
        points INTEGER,
        FOREIGN KEY (match_id)
        REFERENCES matches(id),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        #players
        cursor.execute("""
        CREATE TABLE players(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        team_id INTEGER,
        position VARCHAR(255),
        birthday DATE,
        nationality VARCHAR(255),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        #goals
        cursor.execute("""
        CREATE TABLE goals(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        player_id INTEGER,
        match_id INTEGER,
        goal_type VARCHAR(255),
        FOREIGN KEY (match_id)
        REFERENCES matches(id),
        FOREIGN KEY (player_id)
        REFERENCES players(id)
        )
        ;""")
        conn.commit()


In [2]:
db = ChampionshipsDataBase()

<IPython.core.display.Javascript object>

In [3]:
db.get_tables_names()

['championships',
 'sqlite_sequence',
 'teams',
 'participation',
 'matches',
 'matches_teams',
 'players',
 'goals']

In [4]:
db.get_df('sqlite_sequence')

<IPython.core.display.Javascript object>

,name,seq


#### create championship table sql

In [5]:
#1.championships
db.get_df('championships')
db.cur.execute("""
        INSERT INTO championships(
        name,
        country,
        start_year,
        end_year) VALUES ('Ligue 1', 'France', '2020', '2021');""")



<IPython.core.display.Javascript object>

OperationalError: database is locked

In [ ]:
def insert_row(self, table_name, values):
    db.get_df(table_name)
    

In [6]:
db.get_df('championships')

<IPython.core.display.Javascript object>

,id,name,country,start_year,end_year


#### create teams table in sql

In [7]:
#2.teams
df_teams

NameError: name 'df_teams' is not defined

In [132]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_teams)):
    name = df_teams.loc[i, 'name']
    city = df_teams.loc[i, 'city']
    coach_name = df_teams.loc[i, 'coach_name']
    year_founded = df_teams.loc[i, 'year_founded']
    
    db.cur.execute("""INSERT INTO teams(name, city, coach_name, year_founded) 
SELECT "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM teams WHERE name = "{}" AND city = "{}" AND coach_name = "{}" 
AND year_founded = "{}");"""
                   .format(name, city, coach_name, year_founded, name, city, coach_name, year_founded))       

In [133]:
db.get_df('teams').tail(3)

,id,name,city,coach_name,year_founded
17,18,Stade Rennais Football Club,Rennes,Bruno GENESIO,1901
18,19,Association Sportive de St-Etienne,Saint-Étienne,Claude PUEL,1919
19,20,Racing Club de Strasbourg Alsace,Strasbourg,Julien STEPHAN,1906


#### create participation table  

In [134]:
# insert new row into the table participation preventing dublicates
def update_participation_table(df_teams):
    for i in range(len(df_teams)):
        championship_id = db.cur.execute(
            "SELECT id FROM championships WHERE name = '{}';".format(
                df_teams.loc[i, 'championship'])).fetchone()[0]
        team_id = db.cur.execute(
            "SELECT id FROM teams WHERE city = '{}';".format(
                df_teams.loc[i, 'city'])).fetchone()[0]
        db.cur.execute("""INSERT INTO participation(championship_id, team_id) 
    SELECT '{}', '{}' 
    WHERE NOT EXISTS(SELECT * FROM participation WHERE championship_id = '{}' AND team_id = '{}');"""
                       .format(championship_id, team_id, championship_id,
                               team_id))


update_participation_table(df_teams)

In [135]:
db.get_df('participation').tail(3)

,id,championship_id,team_id
17,18,1,18
18,19,1,19
19,20,1,20


#### create players table 

In [136]:
df_players.head()

,number,name,country,birthday,position,club
0,1.0,P. Bernardoni,FRA,1997-04-18,Gar.,Angers
1,16.0,L. Butelle,FRA,1983-04-03,Gar.,Angers
2,30.0,D. Petkovic,MNE,1993-05-25,Gar.,Angers
3,25.0,A. Bamba,CIV,1990-04-25,Déf.,Angers
4,20.0,K. Boma,FRA,2002-11-20,Déf.,Angers


In [137]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_players)):
    name = df_players.loc[i, 'name']
    team_id = db.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
        df_players.loc[i, 'club'])).fetchone()[0]
    position = df_players.loc[i, 'position']
    birthday = df_players.loc[i, 'birthday']
    nationality = df_players.loc[i, 'country']
    #print(name, team_id, position, birthday, nationality)

    db.cur.execute(
        """INSERT INTO players(name, team_id, position, birthday, nationality) 
            SELECT "{}", "{}", "{}", "{}", "{}" 
            WHERE NOT EXISTS(SELECT * FROM players WHERE name = "{}" AND team_id = "{}" 
            AND position = "{}" AND birthday = "{}" AND nationality = "{}");"""
        .format(name, team_id, position, birthday, nationality, name, team_id,
                position, birthday, nationality))

In [138]:
db.get_df('players')

,id,name,team_id,position,birthday,nationality
0,1,P. Bernardoni,1,Gar.,1997-04-18,FRA
1,2,L. Butelle,1,Gar.,1983-04-03,FRA
2,3,D. Petkovic,1,Gar.,1993-05-25,MNE
3,4,A. Bamba,1,Déf.,1990-04-25,CIV
4,5,K. Boma,1,Déf.,2002-11-20,FRA
5,6,E. Diaw,1,Déf.,1994-12-31,SEN
6,7,S. Doumbia,1,Déf.,1996-09-24,CIV
7,8,E. Ebosse,1,Déf.,1999-03-11,FRA
8,9,V. Manceau,1,Déf.,1989-07-10,FRA
9,10,M. Pavlovic,1,Déf.,1990-06-09,CRO


#### create matches table 

In [139]:
df_matches.tail()

,date,place,temperature,rainfall
375,2021-05-23T21:00:00+0200,Rennes,18,0
376,2021-05-23T21:00:00+0200,Nantes,18,0
377,2021-05-23T21:00:00+0200,Saint-Étienne,18,0
378,2021-05-23T21:00:00+0200,Reims,18,0
379,2021-05-23T21:00:00+0200,Strasbourg,18,0


In [140]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_matches)):
    date = df_matches.loc[i, 'date']
    place = df_matches.loc[i, 'place']
    temperature = df_matches.loc[i, 'temperature']
    rainfall = df_matches.loc[i, 'rainfall']
    
    db.cur.execute("""INSERT INTO matches(date, place, temperature, rainfall) 
SELECT "{}", "{}", "{}", "{}"
WHERE NOT EXISTS(SELECT * FROM matches WHERE date = "{}" AND place = "{}" AND temperature = "{}" 
AND rainfall = "{}");"""
                   .format(date, place, temperature, rainfall, date, place, temperature, rainfall))

In [142]:
db.get_df('matches').shape

(380, 5)

#### create matches_teams table 

In [143]:
df_matches_teams.head()

,date,place,team,team_goal,players
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,0,[]
1,2020-08-21T19:00:00+0200,Bordeaux,Nantes,0,[]
2,2020-08-22T17:00:00+0200,Dijon,Dijon,0,[]
3,2020-08-22T17:00:00+0200,Dijon,Angers,1,[I. Traoré]
4,2020-08-22T21:00:00+0200,Lille,Rennes,1,[D. Da Silva]


In [144]:
# insert new row into the table participation preventing dublicates
for i in range(len(df_matches_teams)):
    match_id = db.cur.execute("SELECT id FROM matches WHERE date = '{}' AND place = '{}';".format(
        df_matches_teams.loc[i, 'date'], df_matches_teams.loc[i, 'place'])).fetchone()[0]
    team_id = db.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
        df_matches_teams.loc[i, 'team'])).fetchone()[0]
    home = df_matches_teams.loc[i, 'team'] == df_matches_teams.loc[i, 'place']
    team_goal = df_matches_teams.loc[i, 'team_goal']
    points = 99
    #print(name, team_id, position, birthday, nationality)
    
    db.cur.execute("""INSERT INTO matches_teams(match_id, team_id, home, team_goal, points) 
SELECT "{}", "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM matches_teams WHERE match_id = "{}" AND team_id = "{}" AND home = "{}" 
AND team_goal = "{}" AND points = "{}");"""
                   .format(match_id, team_id, home, team_goal, points, match_id, team_id, home, team_goal, points))

In [145]:
db.get_df('matches_teams').tail()

,id,match_id,team_id,home,team_goal,points
755,756,373,5,True,0,99
756,757,372,3,True,0,99
757,758,371,1,True,1,99
758,759,379,2,False,2,99
759,760,380,7,False,1,99


#### create goals table 

In [146]:
# insert new row into the table participation preventing dublicates
count = 0
players_not_in_list_names = []
players_not_in_list_clubs = []
for i in range(len(df_matches_teams)):
    club = df_matches_teams.loc[i, 'team']
    for player_name in df_matches_teams.loc[i, 'players']:
        try:
            player_id = db.cur.execute(
                """SELECT id FROM players WHERE name = "{}";""".format(
                    player_name)).fetchone()[0]
        except:
            players_not_in_list_names.append(player_name)
            players_not_in_list_clubs.append(club)
            count += 1
            
print(f'Missing players number - {count}')

# insert new row into the table participation preventing dublicates
for (name, club) in zip(players_not_in_list_names, players_not_in_list_clubs):
    print("{}:{}".format(name, club))
    team_id = db.cur.execute(
        "SELECT id FROM teams WHERE city = '{}';".format(club)).fetchone()[0]
    position = None
    birthday = None
    nationality = None

    db.cur.execute(
        """INSERT INTO players(name, team_id, position, birthday, nationality) 
SELECT "{}", "{}", "{}", "{}", "{}" 
WHERE NOT EXISTS(SELECT * FROM players WHERE name = "{}" AND team_id = "{}" AND position = "{}" 
AND birthday = "{}" AND nationality = "{}");""".format(name, team_id, position,
                                                       birthday, nationality,
                                                       name, team_id, position,
                                                       birthday, nationality))

Missing players number - 18
K. Denkey:Nîmes
A. Scheidler:Dijon
J. Maja:Bordeaux
M. Sanson:Marseille
Raphinha:Rennes
J. Maja:Bordeaux
Pablo:Marseille
K. Lala:Strasbourg
M. Sanson:Marseille
K. Lala:Strasbourg
M. Dembélé:Lyon
M. Niang:Rennes
Pablo:Bordeaux
K. Lala:Strasbourg
N. Radonjic:Marseille
N. Radonjic:Marseille
D. Pereira Da Costa:Lens
Y. Sabaly:Bordeaux


In [149]:
db.get_df('players').shape

(641, 6)

In [150]:
for i in range(len(df_matches_teams)):
    club = df_matches_teams.loc[i, 'team']
    for player_name in df_matches_teams.loc[i, 'players']:
        player_id = db.cur.execute(
            """SELECT id FROM players WHERE name = "{}";""".format(
                player_name)).fetchone()[0]
        match_id = db.cur.execute(
            "SELECT id FROM matches WHERE date = '{}' AND place = '{}';".
            format(df_matches_teams.loc[i, 'date'],
                   df_matches_teams.loc[i, 'place'])).fetchone()[0]
        goal_type = 'Normal'
        db.cur.execute("""INSERT INTO goals(player_id, match_id, goal_type) 
    VALUES ("{}", "{}", "{}" );"""
                       .format(player_id, match_id, goal_type, player_id,
                               match_id, goal_type))

In [151]:
db.get_df('goals').shape

(1049, 4)